# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [29]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,invercargill,-46.4000,168.3500,21.12,59,100,6.71,NZ,1707135410
1,1,alice springs,-23.7000,133.8833,34.75,22,82,4.12,AU,1707135410
2,2,puerto natales,-51.7236,-72.4875,12.25,50,40,13.89,CL,1707135410
3,3,waitangi,-43.9535,-176.5597,16.57,94,38,0.45,NZ,1707135411
4,4,iqaluit,63.7506,-68.5145,-23.15,77,100,2.06,CA,1707135411


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
%%capture --no-display

# Configure the map plot
coordinates= pd.DataFrame(city_data_df)
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [64]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df= city_data_df.loc[(city_data_df['Max Temp']>22)]
city_data_df= city_data_df.loc[(city_data_df['Max Temp']<27)]
city_data_df= city_data_df.loc[(city_data_df['Humidity']>50)]
city_data_df= city_data_df.loc[(city_data_df['Cloudiness']<50)]
city_data_df= city_data_df.loc[(city_data_df['Wind Speed']<3)]
# Drop any rows with null values
city_data_df= city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
19,19,brisas de zicatela,15.8369,-97.0419,23.00,64,0,1.82,MX,1707135413
86,86,port alfred,-33.5906,26.8910,25.10,80,7,2.24,ZA,1707135423
90,90,samana,19.2056,-69.3369,22.12,94,40,0.00,DO,1707135424
104,104,kingston,17.9970,-76.7936,26.53,91,40,2.57,JM,1707135426
125,125,acapulco de juarez,16.8634,-99.8901,22.90,83,0,2.06,MX,1707135136
216,216,razole,16.4833,81.8333,25.70,70,2,2.62,IN,1707135446
224,224,ambon,-3.6954,128.1814,26.08,94,40,1.54,ID,1707135447
258,258,howrah,22.5892,88.3103,25.00,61,1,2.06,IN,1707135453
266,266,plettenberg bay,-34.0527,23.3716,25.60,69,20,2.96,ZA,1707135454
450,450,tainan,22.9908,120.2133,24.93,88,40,1.54,TW,1707135402


### Step 3: Create a new DataFrame called `hotel_df`.

In [65]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df= city_data_df[["City", "Lat", "Lng", "Country", "Humidity"]].copy()

hotel_df =city_data_df.copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]= ""

# Display sample data
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,19,brisas de zicatela,15.8369,-97.0419,23.00,64,0,1.82,MX,1707135413,
86,86,port alfred,-33.5906,26.8910,25.10,80,7,2.24,ZA,1707135423,
90,90,samana,19.2056,-69.3369,22.12,94,40,0.00,DO,1707135424,
104,104,kingston,17.9970,-76.7936,26.53,91,40,2.57,JM,1707135426,
125,125,acapulco de juarez,16.8634,-99.8901,22.90,83,0,2.06,MX,1707135136,
216,216,razole,16.4833,81.8333,25.70,70,2,2.62,IN,1707135446,
224,224,ambon,-3.6954,128.1814,26.08,94,40,1.54,ID,1707135447,
258,258,howrah,22.5892,88.3103,25.00,61,1,2.06,IN,1707135453,
266,266,plettenberg bay,-34.0527,23.3716,25.60,69,20,2.96,ZA,1707135454,
450,450,tainan,22.9908,120.2133,24.93,88,40,1.54,TW,1707135402,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [66]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"hotel",
    "apikey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude= row['Lat']
    longitude= row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
brisas de zicatela - nearest hotel: No hotel found
port alfred - nearest hotel: No hotel found
samana - nearest hotel: No hotel found
kingston - nearest hotel: No hotel found
acapulco de juarez - nearest hotel: No hotel found
razole - nearest hotel: No hotel found
ambon - nearest hotel: No hotel found
howrah - nearest hotel: No hotel found
plettenberg bay - nearest hotel: No hotel found
tainan - nearest hotel: No hotel found
bundaberg - nearest hotel: No hotel found
fort-de-france - nearest hotel: No hotel found
port moresby - nearest hotel: No hotel found
turan - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
19,19,brisas de zicatela,15.8369,-97.0419,23.00,64,0,1.82,MX,1707135413,No hotel found
86,86,port alfred,-33.5906,26.8910,25.10,80,7,2.24,ZA,1707135423,No hotel found
90,90,samana,19.2056,-69.3369,22.12,94,40,0.00,DO,1707135424,No hotel found
104,104,kingston,17.9970,-76.7936,26.53,91,40,2.57,JM,1707135426,No hotel found
125,125,acapulco de juarez,16.8634,-99.8901,22.90,83,0,2.06,MX,1707135136,No hotel found
216,216,razole,16.4833,81.8333,25.70,70,2,2.62,IN,1707135446,No hotel found
224,224,ambon,-3.6954,128.1814,26.08,94,40,1.54,ID,1707135447,No hotel found
258,258,howrah,22.5892,88.3103,25.00,61,1,2.06,IN,1707135453,No hotel found
266,266,plettenberg bay,-34.0527,23.3716,25.60,69,20,2.96,ZA,1707135454,No hotel found
450,450,tainan,22.9908,120.2133,24.93,88,40,1.54,TW,1707135402,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [67]:
%%capture --no-display

# Configure the map plot
#coordinates= pd.DataFrame(hotel_df)
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size ="Humidity",
    tiles = "OSM",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)